In [2]:
from __future__ import division
import random
import sys
import io
import os
import logging
import re
import pandas as pd
import ujson as json
import os.path as op
from tqdm import tqdm
from collections import Counter, OrderedDict
import argparse

In [3]:
entity_linking_pattern = re.compile('#.*?;-*[0-9]+,(-*[0-9]+)#')
fact_pattern = re.compile('#(.*?);-*[0-9]+,-*[0-9]+#')
unk_pattern = re.compile('#([^#]+);-1,-1#')
TSV_DELIM = "\t"
TBL_DELIM = " ; "
def join_unicode(delim, entries):
    #entries = [_.decode('utf8') for _ in entries]
    return delim.join(entries)

def parse_fact(fact):
    fact = re.sub(unk_pattern, '[UNK]', fact)
    chunks = re.split(fact_pattern, fact)
    output = ' '.join([x.strip() for x in chunks if len(x.strip()) > 0])
    return output

In [4]:
import os

table_path = './data/tables'
output_path = './data/tabfact_process'
if not os.path.exists(output_path):
    os.mkdir(output_path)

table_data = []
for file in tqdm(os.listdir(table_path)):
    file_path = os.path.join(table_path, file)
    with open(file_path, 'r') as f:
        table_file = json.load(f)
        table_file['file_name'] = file
        table_data.append(table_file)
    break

  0%|          | 0/137824 [00:00<?, ?it/s]


In [24]:
def tabfact_template(tablex, scan = 'horizontal'):
    #print(tablex['file_name'])
    table_header = tablex['header']
    datax = tablex['data']
    n_row, n_column = len(datax), len(datax[0])
    #print(n_row, n_column)
    table_cells = []
    if scan == 'horizontal':
        for i in range(n_row):
            table_cells.append('row {} is :'.format(i + 1))
            this_row = []
            for j in range(n_column):
                this_row.append('{} is {}'.format(table_header[j][0], datax[i][j][0]))
            this_row = join_unicode(TBL_DELIM, this_row)
            table_cells.append(this_row)
            table_cells.append('.')
    else:
        pass

    table_str = ' '.join(table_cells)
    #print(table_str)
    results = {'table_text': table_str}
    for k, v in tablex.items():
        results[k] = v
    return results

tabfact_template(table_data[0])

{'table_text': 'row 1 is : Pick is 44 ; Player is Parker Meadows ; Team is Detroit Tigers ; Position is Outfielder ; School is Grayson High School (GA) . row 2 is : Pick is 45 ; Player is Sean Hjelle ; Team is San Francisco Giants ; Position is Pitcher ; School is Kentucky . row 3 is : Pick is 46 ; Player is Steele Walker ; Team is Chicago White Sox ; Position is Outfielder ; School is Oklahoma . row 4 is : Pick is 52 ; Player is Griffin Conine ; Team is Toronto Blue Jays ; Position is Outfielder ; School is Duke . row 5 is : Pick is 56 ; Player is Tyler Frank ; Team is Tampa Bay Rays ; Position is Shortstop ; School is Florida Atlantic . row 6 is : Pick is 65 ; Player is Tim Cate ; Team is Washington Nationals ; Position is Pitcher ; School is Connecticut . row 7 is : Pick is 67 ; Player is Nick Sandlin ; Team is Cleveland Indians ; Position is Pitcher ; School is Southern Miss . row 8 is : Pick is 70 ; Player is Jeremy Eierman ; Team is Oakland Athletics ; Position is Shortstop ; Sch

In [28]:
from multiprocessing import Pool
import multiprocessing
import json
cores = multiprocessing.cpu_count()
pool = Pool(cores)
rs = pool.map(tabfact_template, table_data)

for i in tqdm(range(len(rs))):
    tablex = rs[i]
    file_name = tablex['file_name']
    file_path = os.path.join(output_path, file_name)
    with open(file_path, 'w') as f:
        json.dump(tablex, f)

100%|██████████| 1/1 [00:00<00:00, 59.20it/s]


In [10]:
import jsonlines
import json
import gzip
from tqdm import tqdm
filename = '/data1/fch123/OTT-QA/data/v1.0/train/nq-train-01.jsonl.gz'
train_data = []
with gzip.open(filename, 'r') as f:
    print('start load json data')
    for i in tqdm(f):
        train_data.append(json.loads(i))
        break

for k, v in train_data[0].items():
    print(k)

start load json data


0it [00:00, ?it/s]

annotations
document_html
document_title
document_tokens
document_url
example_id
long_answer_candidates
question_text
question_tokens


In [6]:
from bs4 import BeautifulSoup#
#print(train_data[0]['document_tokens'])
#print(train_data[0]['long_answer_candidates'])
#print(len(train_data[0]['document_html']))

soup = BeautifulSoup(train_data[0]['document_html'], 'html.parser')
tmp = soup.find_all(class_='wikitable')
rs = []
rest = []
for _ in tmp:
    if _['class'] == ['wikitable', 'sortable']:
        rs.append(_)
    else:
        rest.append(_)
print(rest[0].find_all('tr'))


[<tr style="color:white;text-align:center">
<th scope="col" style="background:#555752;width:5%"><abbr title="Number">No.</abbr><br/>
overall</th>
<th scope="col" style="background:#555752;width:5%"><abbr title="Number">No.</abbr> in<br/>
season</th>
<th scope="col" style="background:#555752;width:23%">Title</th>
<th scope="col" style="background:#555752;width:16%">Directed by</th>
<th scope="col" style="background:#555752;width:27%">Written by</th>
<th scope="col" style="background:#555752;width:13%">Original air date</th>
<th scope="col" style="background:#555752;width:10%">U.S. viewers<br/>
(millions)</th>
</tr>, <tr class="vevent" style="text-align:center;background:#F2F2F2">
<th id="ep100" scope="row" style="text-align:center">100</th>
<td>1</td>
<td class="summary" style="text-align:left">"<a href="/wiki/Mercy_(The_Walking_Dead)" title="Mercy (The Walking Dead)">Mercy</a>"</td>
<td><a href="/wiki/Greg_Nicotero" title="Greg Nicotero">Greg Nicotero</a></td>
<td><a href="/wiki/Scott_

In [ ]:
def convert_to_tsv(out_file, examples, dataset_type, meta, scan):

    total = 0
    unk = 0
    len_total = 0
    empty_table = 0
    with io.open(out_file, 'w', encoding='utf-8') as fout:
        for example in tqdm(examples):
            assert len(example['facts']) == len(example['labels'])
            for fact, label in zip(example['facts'], example['labels']):
                # use entity linking info to retain relevant columns
                useful_column_nums = [int(x) for x in re.findall(entity_linking_pattern, fact) if not x == '-1']
                useful_column_nums = dict.fromkeys(useful_column_nums)
                remaining_table = OrderedDict()
                for idx, (column_type, column_cells) in enumerate(example['columns'].items()):
                    if idx in useful_column_nums:
                        column_type = '_'.join(column_type.split())
                        remaining_table[column_type] = column_cells

                fact_clean = parse_fact(fact)
                if len(remaining_table) > 0:
                    table_cells, table_feats = [], []

                    len_total += 1
                    if scan == 'vertical':
                        for column_type, column_cells in remaining_table.items():
                            column_type = ' '.join(column_type.split('_'))
                            table_cells.extend([column_type, 'are :'])
                            this_column = []
                            for idx, c in enumerate(column_cells):
                                this_column.append("row {} is {}".format(idx + 1, c))
                            this_column = join_unicode(TBL_DELIM, this_column)
                            table_cells.append(this_column)
                            table_cells.append('.')
                            table_feats.append(column_type)
                    else:
                        # stupid but to reserve order
                        table_column_names, table_column_cells = [], []
                        for column_type, column_cells in remaining_table.items():
                            column_type = ' '.join(column_type.split('_'))
                            table_feats.append(column_type)
                            table_column_names.append(column_type)
                            table_column_cells.append(column_cells)
                        for idx, row in enumerate(zip(*table_column_cells)):
                            table_cells.append('row {} is :'.format(idx + 1))
                            this_row = []
                            for col, tk in zip(table_column_names, row):
                                this_row.append('{} is {}'.format(col, tk))
                            this_row = join_unicode(TBL_DELIM, this_row)
                            table_cells.append(this_row)
                            table_cells.append('.')

                    table_str = ' '.join(table_cells)
                    out_items = [example['csv'],
                                 str(len(table_feats)),
                                 ' '.join([str(x) for x in table_feats]),
                                 table_str,
                                 fact_clean,
                                 str(label)]

                    out_items = TSV_DELIM.join(out_items)
                    total += 1
                    fout.write(out_items + "\n")
                else:
                    if dataset_type != 'train':
                        table_feats = ['[UNK]']
                        table_cells = ['[UNK]']
                        table_str = ' '.join(table_cells)
                        out_items = [example['csv'],
                                     str(len(table_feats)),
                                     ' '.join([str(x) for x in table_feats]),
                                     table_str,
                                     fact_clean,
                                     str(label)]

                        out_items = TSV_DELIM.join(out_items)
                        fout.write(out_items + "\n")
                        total += 1
                    empty_table += 1
    print("Built {} instances of features in total, {}/{}={}% unseen column types, {} empty tables"
          .format(total, unk, len_total, "{0:.2f}".format(unk * 100 / len_total), empty_table))
    meta["{}_total".format(dataset_type)] = total

    return meta


    def convert_to_tsv(out_file, examples, dataset_type, meta, scan):
        